In [2]:
export cwd=$(pwd)

In [3]:
echo $cwd

/cluster/gs_lab/common/co_chip


In [3]:
export SRR_RUN_TABLE=$cwd/SraRunTable.txt

In [4]:
cd $cwd
mkdir run
cd run

mkdir: cannot create directory `run': File exists


In [5]:
rm -rf *

In [6]:
mkdir index
mkdir annotation

In [ ]:
cd annotation
wget ftp://ftp.ensembl.org/pub/release-97/fasta/mus_musculus/dna//Mus_musculus.GRCm38.dna.toplevel.fa.gz
cd ..

--2019-08-16 11:00:12--  ftp://ftp.ensembl.org/pub/release-97/fasta/mus_musculus/dna//Mus_musculus.GRCm38.dna.toplevel.fa.gz
           => 'Mus_musculus.GRCm38.dna.toplevel.fa.gz'
Resolving ftp.ensembl.org... 193.62.193.8
Connecting to ftp.ensembl.org|193.62.193.8|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/release-97/fasta/mus_musculus/dna/ ... done.
==> SIZE Mus_musculus.GRCm38.dna.toplevel.fa.gz ... 842816838
==> PASV ... done.    ==> RETR Mus_musculus.GRCm38.dna.toplevel.fa.gz ... done.
Length: 842816838 (804M) (unauthoritative)



In [ ]:
cd index
bowtie2-build ../annotation/Mus_musculus.GRCm38.dna.toplevel.fa.gz mm9
bowtie2-inspect -n mm9
export INDEX=$(pwd)/mm9
cd ..

In [ ]:
python3 $cwd/parse_sra_runtable.py $SRR_RUN_TABLE

In [ ]:
find ./*_*/SRX* -mindepth 0 -maxdepth 0 | parallel -j 17 "$cwd/align.sh {} 2"

In [7]:
find ./*_*/SRX*/merged -mindepth 0 -maxdepth 0 | parallel -j 40 $'samtools view -h {}/all.bam |awk "sqrt((\$9)^2) < 230" |\samtools view -bS - > {}/all-filtered.bam'

In [8]:
find ./*_*/SRX*/merged -mindepth 0 -maxdepth 0 | parallel -j 40 "samtools index {}/all-filtered.bam"

In [9]:
find ./*_*/SRX*/merged -mindepth 0 -maxdepth 0 | parallel -j 17 "bamPEFragmentSize -b {}/all-filtered.bam -o {}/hist.png --numberOfProcessors 2"



BAM file : ./H3K27ac_H3K27ac/SRX1943296/merged/all-filtered.bam
Sample size: 10401

Fragment lengths:
Min.: 0.0
1st Qu.: 113.0
Mean: 147.74896644553408
Median: 151.0
3rd Qu.: 187.0
Max.: 229.0
Std: 48.63484644531806
MAD: 37.0
Len. 10%: 83.0
Len. 20%: 104.0
Len. 30%: 122.0
Len. 40%: 136.0
Len. 60%: 166.0
Len. 70%: 179.0
Len. 80%: 194.0
Len. 90%: 211.0
Len. 99%: 228.0


Read lengths:
Sample size: 10401

Min.: 38.0
1st Qu.: 38.0
Mean: 38.0
Median: 38.0
3rd Qu.: 38.0
Max.: 38.0
Std: 0.0
MAD: 0.0
Len. 10%: 38.0
Len. 20%: 38.0
Len. 30%: 38.0
Len. 40%: 38.0
Len. 60%: 38.0
Len. 70%: 38.0
Len. 80%: 38.0
Len. 90%: 38.0
Len. 99%: 38.0

/homes/dhelekal/anaconda3/envs/experiment_env/lib/python3.5/site-packages/matplotlib/axes/_axes.py:6499: MatplotlibDeprecationWarning:


The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.



BAM file : ./H3K27ac_H3K4me3/SRX1943227/merged/all-filtered.bam
Sample size: 7781

Fragment lengths:
Min.: 0.0
1st Qu.: 11

Mean: 149.77903298689563
Median: 155.0
3rd Qu.: 189.0
Max.: 229.0
Std: 50.03804803068047
MAD: 37.0
Len. 10%: 83.0
Len. 20%: 107.0
Len. 30%: 124.0
Len. 40%: 140.0
Len. 60%: 169.0
Len. 70%: 183.0
Len. 80%: 197.0
Len. 90%: 213.0
Len. 99%: 228.0


Read lengths:
Sample size: 15491

Min.: 38.0
1st Qu.: 38.0
Mean: 38.0
Median: 38.0
3rd Qu.: 38.0
Max.: 38.0
Std: 0.0
MAD: 0.0
Len. 10%: 38.0
Len. 20%: 38.0
Len. 30%: 38.0
Len. 40%: 38.0
Len. 60%: 38.0
Len. 70%: 38.0
Len. 80%: 38.0
Len. 90%: 38.0
Len. 99%: 38.0

/homes/dhelekal/anaconda3/envs/experiment_env/lib/python3.5/site-packages/matplotlib/axes/_axes.py:6499: MatplotlibDeprecationWarning:


The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.



BAM file : ./H3K27ac_H3K4me3/SRX1943226/merged/all-filtered.bam
Sample size: 5515

Fragment lengths:
Min.: 0.0
1st Qu.: 108.0
Mean: 143.0391659111514
Median: 145.0
3rd Qu.: 184.0
Max.: 229.0
Std: 50.39914579310946
MAD: 38.0
Len. 10%: 79.0
Len. 20%: 99

In [10]:
find ./*_*/SRX* -mindepth 0 -maxdepth 0 | parallel -j 40 "makeTagDirectory {}/tag_dir {}/merged/all-filtered.bam"

	Will parse file: ./H3K27ac_H3K4me3/SRX1943198/merged/all-filtered.bam

	Creating directory: ./H3K27ac_H3K4me3/SRX1943198/tag_dir and removing existing *.tags.tsv

	Treating ./H3K27ac_H3K4me3/SRX1943198/merged/all-filtered.bam as a bam file
	Reading alignment file ./H3K27ac_H3K4me3/SRX1943198/merged/all-filtered.bam

	Optimizing tag files...
	Estimated genome size = 6321424879
	Estimated average read density = 0.000380 per bp
	Total Tags = 2399287.0
	Total Positions = 4084962
	Average tag length = 38.1
	Median tags per position = 0 (ideal: 1)
	Average tags per position = 0.154
	Fragment Length Estimate: 155
	Peak Width Estimate: 141
		!!! No reliable estimate for peak size
		Setting Peak width estimate to be equal to fragment length estimate
	Autocorrelation quality control metrics:
		Same strand fold enrichment: 2.0
		Diff strand fold enrichment: 2.2
		Same / Diff fold enrichment: 0.9

		Guessing sample is ChIP-Seq or unstranded RNA-Seq - autocorrelation looks good.

	Will parse file:

	Fragment Length Estimate: 149
	Peak Width Estimate: 147
		!!! No reliable estimate for peak size
		Setting Peak width estimate to be equal to fragment length estimate
	Autocorrelation quality control metrics:
		Same strand fold enrichment: 1.6
		Diff strand fold enrichment: 1.7
		Same / Diff fold enrichment: 0.9

		Guessing sample is ChIP-Seq or unstranded RNA-Seq - autocorrelation looks good.

	Will parse file: ./H3K27ac_H3K27ac/SRX1943295/merged/all-filtered.bam

	Creating directory: ./H3K27ac_H3K27ac/SRX1943295/tag_dir and removing existing *.tags.tsv

	Treating ./H3K27ac_H3K27ac/SRX1943295/merged/all-filtered.bam as a bam file
	Reading alignment file ./H3K27ac_H3K27ac/SRX1943295/merged/all-filtered.bam

	Optimizing tag files...
	Estimated genome size = 7264206149
	Estimated average read density = 0.000977 per bp
	Total Tags = 7096986.5
	Total Positions = 12197120
	Average tag length = 38.0
	Median tags per position = 0 (ideal: 1)
	Average tags per position = 0.144
	Fragment Length

In [ ]:
find ./*_*/SRX* -mindepth 0 -maxdepth 0 | parallel -j 40 "findPeaks {}/tag_dir -style histone -size 200 -minDist 100 -o auto"